# Oscilloskope utility – using Ethernet

In [17]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run "Record" on the oscilloscope
# and wait for 508 frames to be acquired.
# Step 2:
# Run this cell to initialize grabbing.


# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
        
# Default oscilloscope record timeout [s]
loop_sleep_time = 60
        
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::10.9.9.100::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::10.9.9.100::INSTR
RIGOL TECHNOLOGIES,DS2072A,DS2D204202923,00.03.05.SP4


## Read repeatedly records from oscilloscope
This should be run after the initialization step. Timeout at the end should be enlarged if not all 508 frames are transferred.

In [14]:
import os
import glob
import time
import h5py
import numpy as np

filename = 1
run_start_time = time.time()

if filename == 1:
    for f in glob.iglob("./data2/*.h5"):  # delete all .h5 files 
        print('Deleting', f)
        os.remove(f)
else:
    print('Not removing old files, as filename {} is not 1.'.format(filename))

while True:
    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print('XINC:', xinc, end=' ')
    
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print('YINC:', yinc, end=' ')
    
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print('TRIG:', trig, end=' ')
    
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print('YORIGIN:', yorig, end=' ')
    
    osc.write(':WAVeform:XORigin?')
    xorig = float(osc.read(100))
    print('XORIGIN:', xorig, end=' ')

    osc.write(':FUNC:WREP:FEND?')  # get number of last frame
    frames = int(osc.read(100))
    print('FRAMES:', frames, 'SUBRUN', filename)

    with h5py.File(f'./data/t{filename:02.0f}.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames))
        hf.create_dataset('XINC', data=(xinc))
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        hf.create_dataset('XORIGIN', data=(xorig))

        osc.write(':FUNC:WREP:FCUR 1')  # skip to first frame
        time.sleep(0.5)

        for n in range(1, frames + 1):
            osc.write(f':FUNC:WREP:FCUR {n}')
            time.sleep(0.001)

            osc.write(':WAV:DATA?')  # read data
            wave1 = bytearray(osc.read_raw(500))
            wave2 = bytearray(osc.read_raw(500))
            wave3 = bytearray(osc.read_raw(500))

            wave = np.concatenate((wave1[11:], wave2, wave3[:-1]))
            hf.create_dataset(str(n), data=wave)

    filename += 1
    osc.write(':FUNC:WREC:OPER REC')  # start recording

    print('  Subrun finished, press Enter to continue.')
    input()


XINC: 5e-07 YINC: 0.0004 TRIG: -0.01 YORIGIN: 47.0 XORIGIN: -0.00035 FRAMES: 18 SUBRUN 1
  Subrun finished, press Enter to continue.

XINC: 5e-07 YINC: 0.0004 TRIG: -0.01 YORIGIN: 47.0 XORIGIN: -0.00035 FRAMES: 0 SUBRUN 2
  Subrun finished, press Enter to continue.

XINC: 5e-07 YINC: 0.0004 TRIG: -0.01 YORIGIN: 47.0 XORIGIN: -0.00035 FRAMES: 0 SUBRUN 3
  Subrun finished, press Enter to continue.


KeyboardInterrupt: Interrupted by user

### Stopwatch for timing the first loop

In [ ]:
first_run_start_time = time.time()
raw_input()
loop_sleep_time = time.time() - first_run_start_time + 15
print loop_sleep_time

In [9]:
loop_sleep_time=60

In [19]:
import os
import glob
import time
import h5py
import numpy as np
import sys

filename = 1

if filename == 1:
    for f in glob.iglob("./data/*.h5"):
        print('Deleting', f)
        os.remove(f)
else:
    print('Not removing old files, as filename {} is not 1.'.format(filename))

osc.write(':STOP')
time.sleep(0.5)

start_wfd = 0.005
wfd = start_wfd

while True:
    osc.write(':FUNC:WREC:OPER REC')
    run_start_time = time.time()
    print('  Capturing...')
    time.sleep(0.5)

    while True:
        osc.write(':FUNC:WREC:OPER?')
        reply = osc.read()
        if reply == 'STOP':
            run_time = round(time.time() - run_start_time, 2)
            print('  Subrun finished, capturing for {:.2f} seconds.'.format(run_time))
            break
        time.sleep(0.01)

    channel = 'CHAN1'
    if osc.ask(':' + channel + ':DISP?') == '0':
        print(channel + ' is not enabled')
    else:
        print('Reading out ' + channel)
        osc.write(':WAV:SOUR ' + channel)
        osc.write(':WAV:MODE NORM')
        osc.write(':WAV:FORM BYTE')
        osc.write(':WAV:POIN 1400')

        osc.write(':WAV:XINC?')
        xinc = float(osc.read(100))
        print('XINC:', xinc)
        osc.write(':WAV:YINC?')
        yinc = float(osc.read(100))
        print('YINC:', yinc)
        osc.write(':TRIGger:EDGe:LEVel?')
        trig = float(osc.read(100))
        print('TRIG:', trig)
        osc.write(':WAVeform:YORigin?')
        yorig = float(osc.read(100))
        print('YORIGIN:', yorig)
        osc.write(':WAVeform:XORigin?')
        xorig = float(osc.read(100))
        print('XORIGIN:', xorig)
        osc.write(':FUNC:WREP:FEND?')
        frames = int(osc.read(100))
        print('FRAMES:', frames, 'SUBRUN', filename)

        lastwave = bytearray()

        with h5py.File(f'./data/data{filename:02d}_{int(round(run_start_time, 0))}-{channel}.h5', 'w') as hf:
            hf.create_dataset('FRAMES', data=frames)
            hf.create_dataset('XINC', data=xinc)
            hf.create_dataset('YINC', data=yinc)
            hf.create_dataset('TRIG', data=trig)
            hf.create_dataset('YORIGIN', data=yorig)
            hf.create_dataset('XORIGIN', data=xorig)
            hf.create_dataset('CAPTURING', data=run_time)
            osc.write(':FUNC:WREP:FCUR 1')
            time.sleep(0.5)

            for n in range(1, frames + 1):
                osc.write(f':FUNC:WREP:FCUR {n}')
                while True:
                    time.sleep(0.05)
                    fcur = osc.ask(':FUNC:WREP:FCUR?')
                    if str(n) == fcur:
                        sys.stdout.write(str(n))
                        break
                    else:
                        print(f"Needwait: {n} vs {fcur}")

                reread_count = 0
                while True:
                    time.sleep(wfd)
                    osc.write(':WAV:DATA?')
                    time.sleep(wfd)
                    wave1 = bytearray(osc.read_raw(500))
                    wave2 = bytearray(osc.read_raw(500))
                    wave3 = bytearray(osc.read_raw(500))
                    wave = np.concatenate((wave1[11:], wave2, wave3[:-1]))

                    if np.array_equal(wave, lastwave):
                        wfd += 0.005
                        print(f' Same waveform, wait {wfd:.3f} and reread')
                        reread_count += 1
                        if reread_count > 5:
                            print('------------ Wrong trigger level?')
                    else:
                        hf.create_dataset(str(n), data=wave)
                        lastwave = wave
                        sys.stdout.write('.')
                        wfd = start_wfd
                        break

        print('OK')
    filename += 1


  Capturing...
  Subrun finished, capturing for 15.88 seconds.
Reading out CHAN1
XINC: 5e-07
YINC: 0.0004
TRIG: -0.0054
YORIGIN: 47.0
XORIGIN: -0.00035
FRAMES: 63 SUBRUN 1
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.OK
  Capturing...
  Subrun finished, capturing for 24.00 seconds.
Reading out CHAN1
XINC: 5e-07
YINC: 0.0004
TRIG: -0.0054
YORIGIN: 47.0
XORIGIN: -0.00035
FRAMES: 63 SUBRUN 2
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.OK
  Capturing...


KeyboardInterrupt: 